In [14]:
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from dataset import SupportDataset
import numpy as np
import pandas as pd

# Support2

In [15]:
df = pd.read_csv('../data/support2.csv')

In [16]:
data = SupportDataset(df)

/mnt/c/Users/rizki/Master Files/ThesisUMCG/Survival-Tree-Analysis/src/dataset.py:94: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.data.replace({False: 0, True: 1}, inplace=True)


In [17]:
d = data.get_data()
d

array([[62.84998,  0.     ,  1.     , ...,  1.     ,  0.     ,  0.     ],
       [60.33899,  2.     ,  3.     , ...,  0.     ,  1.     ,  0.     ],
       [52.74698,  2.     ,  4.     , ...,  0.     ,  1.     ,  0.     ],
       ...,
       [70.38196,  1.     , 18.     , ...,  0.     ,  1.     ,  0.     ],
       [47.01999,  1.     , 22.     , ...,  0.     ,  0.     ,  1.     ],
       [81.53894,  1.     ,  1.     , ...,  0.     ,  1.     ,  0.     ]])

In [18]:
X_train, X_test, y_train, y_test = data.get_train_test_xgboost()

In [19]:
dtrain = xgb.DMatrix(X_train)
dtrain.set_float_info('label_lower_bound', y_train['Survival_label_lower_bound'])
dtrain.set_float_info('label_upper_bound', y_train['Survival_label_upper_bound'])

In [20]:
dvalid = xgb.DMatrix(X_test)
dvalid.set_float_info('label_lower_bound', y_test['Survival_label_lower_bound'])
dvalid.set_float_info('label_upper_bound', y_test['Survival_label_upper_bound'])

In [21]:
params = {'verbosity': 0,
          'objective': 'survival:aft',
          'eval_metric': 'aft-nloglik',
          'tree_method': 'hist',
          'learning_rate': 0.05,
          'aft_loss_distribution': 'normal',
          'aft_loss_distribution_scale': 1.2,
          'max_depth': 6,
          'lambda': 0.01,
          'alpha': 0.02}

bst = xgb.train(params, dtrain, num_boost_round=1000,
                evals=[(dtrain, 'train'), (dvalid, 'valid')],
                early_stopping_rounds=10)

[0]	train-aft-nloglik:15.00177	valid-aft-nloglik:15.30634
[1]	train-aft-nloglik:14.05325	valid-aft-nloglik:14.35721


[2]	train-aft-nloglik:13.10038	valid-aft-nloglik:13.41016
[3]	train-aft-nloglik:12.18803	valid-aft-nloglik:12.49071
[4]	train-aft-nloglik:11.35780	valid-aft-nloglik:11.65341
[5]	train-aft-nloglik:10.60332	valid-aft-nloglik:10.89063
[6]	train-aft-nloglik:9.91694	valid-aft-nloglik:10.19749
[7]	train-aft-nloglik:9.29235	valid-aft-nloglik:9.56679
[8]	train-aft-nloglik:8.72220	valid-aft-nloglik:8.98999
[9]	train-aft-nloglik:8.20292	valid-aft-nloglik:8.46526
[10]	train-aft-nloglik:7.72934	valid-aft-nloglik:7.98467
[11]	train-aft-nloglik:7.29714	valid-aft-nloglik:7.54617
[12]	train-aft-nloglik:6.90242	valid-aft-nloglik:7.14511
[13]	train-aft-nloglik:6.54185	valid-aft-nloglik:6.77727
[14]	train-aft-nloglik:6.21188	valid-aft-nloglik:6.44134
[15]	train-aft-nloglik:5.91021	valid-aft-nloglik:6.13329
[16]	train-aft-nloglik:5.63362	valid-aft-nloglik:5.85188
[17]	train-aft-nloglik:5.38037	valid-aft-nloglik:5.59375
[18]	train-aft-nloglik:5.14820	valid-aft-nloglik:5.35673
[19]	train-aft-nloglik:4.93527

In [22]:
times_pred = bst.predict(dvalid, output_margin=True)
times_pred

array([6.733583 , 6.653086 , 7.761915 , ..., 7.026075 , 7.7265673,
       7.647699 ], dtype=float32)

In [23]:
times_true = []
event_true = []
for idx, rows in y_test.iterrows():
    if rows['Survival_label_upper_bound'] == np.inf or not rows['Survival_label_upper_bound']:
        event = 0
    else:
        event = 1 
    event_true.append(event)
    times = rows['Survival_label_lower_bound']
    times_true.append(times)
times_true = np.array(times_true)
event_true = np.array(event_true)

In [24]:
y_test.head()

,Survival_label_lower_bound,Survival_label_upper_bound
8630,109,inf
1069,866,866.0
6549,128,inf
5459,35,inf
5318,582,582.0


In [25]:
from lifelines.utils import concordance_index
c_index = concordance_index(times_true, times_pred, event_true)
c_index

0.7553952338009053